El objetivo de esta entrega es extraer los datos de la API de yahoo finance que me permitan evaluar los precios de cierre de las acciones denominadas las 7 Magnificas. Una vez obtenidos los datos cargarlos a Amazon Redshift

In [23]:
#pip install yfinance

#Importo Bibliotecas Basicas
import pandas as pd
import numpy as np
#importar yfinance (Previa instalacion de la API: pip install yfinance) para tener datos de mercado financiero
import yfinance as yf
#Importar librerias de Visualizacion
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns


# 1) Obtengo los registros de la API

In [24]:
#1.1) Creo una lista en la que incluyo los tickers de las Acciones denominadas 7 Maginifcas
tickers = ['MSFT','GOOG','AAPL','AMZN', 'META', 'TSLA', 'NVDA']

# Descargo el precio de Cierre de las Acciones antes mencionada
data = {}
for ticker in tickers:
    data[ticker] = yf.download(ticker, period="10Y", interval="1d")['Close']

# Creo DataFrame 
df = pd.DataFrame(data
                  )
#Imprimo el DataFrame
print(df)
print(df.columns)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

                  MSFT        GOOG        AAPL        AMZN        META  \
Date                                                                     
2014-05-07   39.419998   25.428186   21.154642   14.635500   57.389999   
2014-05-08   39.639999   25.480045   20.999643   14.416000   56.759998   
2014-05-09   39.540001   25.865486   20.912144   14.612000   57.240002   
2014-05-12   39.970001   26.423454   21.172501   15.143000   59.830002   
2014-05-13   40.419998   26.581520   21.205713   15.232000   59.830002   
...                ...         ...         ...         ...         ...   
2024-04-30  389.329987  164.639999  170.330002  175.000000  430.170013   
2024-05-01  394.940002  165.570007  169.300003  179.000000  439.190002   
2024-05-02  397.839996  168.460007  173.029999  184.720001  441.679993   
2024-05-03  406.660004  168.990005  183.380005  186.210007  451.959991   
2024-05-06  413.540009  169.830002  181.710007  188.699997  465.679993   

                  TSLA        NVDA  


In [25]:
# Mover el índice a la primera columna y reorganizar las columnas
df.reset_index(inplace=True)  # Reiniciar el índice y mover Date a una columna
df = df[['Date', 'MSFT', 'GOOG', 'AAPL', 'AMZN', 'META', 'TSLA', 'NVDA']]  # Reorganizar las columnas según sea necesario
df['Date'] = pd.to_datetime(df['Date'])  # Convertir la columna Date a tipo datetime

#Imprimo el DataFrame, sus columnas y tipos de datos contenidos en las mismas
print(df)
print(df.columns)
print(df.dtypes)

           Date        MSFT        GOOG        AAPL        AMZN        META  \
0    2014-05-07   39.419998   25.428186   21.154642   14.635500   57.389999   
1    2014-05-08   39.639999   25.480045   20.999643   14.416000   56.759998   
2    2014-05-09   39.540001   25.865486   20.912144   14.612000   57.240002   
3    2014-05-12   39.970001   26.423454   21.172501   15.143000   59.830002   
4    2014-05-13   40.419998   26.581520   21.205713   15.232000   59.830002   
...         ...         ...         ...         ...         ...         ...   
2512 2024-04-30  389.329987  164.639999  170.330002  175.000000  430.170013   
2513 2024-05-01  394.940002  165.570007  169.300003  179.000000  439.190002   
2514 2024-05-02  397.839996  168.460007  173.029999  184.720001  441.679993   
2515 2024-05-03  406.660004  168.990005  183.380005  186.210007  451.959991   
2516 2024-05-06  413.540009  169.830002  181.710007  188.699997  465.679993   

            TSLA        NVDA  
0      13.423333    

In [26]:
#Cargo el DataFrame como CSV
df.to_csv('Entregable1_Santiago_Hourcade.csv', index=False)

# 2) Cargo la tabla a Redshift

In [27]:
#2.1) Creando la conexión a Redsshift
import psycopg2
host="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
data_base="data-engineer-database"
user="cpn_santiago_hourcade_coderhouse"
with open("pwd_redshift.txt",'r') as f:
    pwd= f.read()
try:
    conn = psycopg2.connect(
        host=host,
        dbname=data_base,
        user=user,
        password=pwd,
        port='5439'
    )
    print("Conectado a Redshift con éxito!")
    
except Exception as e:
    print("No es posible conectar a Redshift")
    print(e)


Conectado a Redshift con éxito!


In [28]:
#2.2) Crear la tabla si no existe
with conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS cpn_santiago_hourcade_coderhouse.siete_magnificas
        (
        Date DATE PRIMARY KEY,
        MSFT VARCHAR(100),
        GOOG VARCHAR(100),
        AAPL VARCHAR(100),
        AMZN VARCHAR(100),
        META VARCHAR(100),
        TSLA VARCHAR(100),
        NVDA VARCHAR(100)     
        )
    """)
    conn.commit()

In [29]:
#2.2.1) Vaciar la tabla para evitar duplicados o inconsistencias
with conn.cursor() as cur:
  cur.execute("Truncate table siete_magnificas")
  count = cur.rowcount


In [30]:
#2.2.2) Consultando que la tabla este vacia
cur = conn.cursor()
cur.execute("SELECT * FROM cpn_santiago_hourcade_coderhouse.siete_magnificas")
results = cur.fetchall()
print(results)


[]


In [31]:
#2.3) Insertando los datos en Redsfhift
from psycopg2.extras import execute_values
with conn.cursor() as cur:
    execute_values(
        cur,
        '''
        INSERT INTO siete_magnificas (Date, MSFT, GOOG, AAPL, AMZN, META, TSLA, NVDA)
        VALUES %s
        ''',
        [tuple(row) for row in df.values],
        page_size=len(df)
    )
    conn.commit()

In [32]:
#2.4) Consultando los registros cargados en la tabla
cur = conn.cursor()
cur.execute("SELECT * FROM cpn_santiago_hourcade_coderhouse.siete_magnificas")
results = cur.fetchall()
print(results)


[(datetime.date(2014, 5, 7), '39.41999816894531', '25.428186416625977', '21.15464210510254', '14.635499954223633', '57.38999938964844', '13.423333168029785', '4.567500114440918'), (datetime.date(2014, 5, 8), '39.63999938964844', '25.480045318603516', '20.999643325805664', '14.416000366210938', '56.7599983215332', '11.906000137329102', '4.625'), (datetime.date(2014, 5, 9), '39.540000915527344', '25.86548614501953', '20.91214370727539', '14.612000465393066', '57.2400016784668', '12.150667190551758', '4.512499809265137'), (datetime.date(2014, 5, 12), '39.970001220703125', '26.42345428466797', '21.172500610351562', '15.142999649047852', '59.83000183105469', '12.311332702636719', '4.644999980926514'), (datetime.date(2014, 5, 13), '40.41999816894531', '26.581520080566406', '21.205713272094727', '15.232000350952148', '59.83000183105469', '12.677332878112793', '4.570000171661377'), (datetime.date(2014, 5, 14), '40.2400016784668', '26.26040267944336', '21.20964241027832', '14.880999565124512', 

In [33]:
#2.5) Cierro Conexion
conn.close()
cur.close()